In [1]:
import pandas as pd 
import numpy as np 
from itertools import product

In [2]:
countries = ['us','de','dk','fr','it','nl','se','sp']

In [3]:
test = pd.read_pickle('output/params_ref_us.pkl')
test

,value,free,se
name,,,
sigma,2.097183,True,0.010304
beta,0.970000,False,NaN
phi,0.304140,True,0.135992
psi,0.168902,True,0.019482
delta_h1,-0.967380,True,0.160692
delta_h2,3.487244,True,0.051008
eta,0.000000,False,NaN
tfp,1.000000,False,NaN
price,1.000000,False,NaN


In [4]:
pars_select = ['delta_h1','delta_h2','price','tfp']
stat = ['par','se']
tuples = list(product(*[pars_select,stat]))
table = pd.DataFrame(index=pd.MultiIndex.from_tuples(tuples),columns=countries)
for c in countries: 
	df = pd.read_pickle('output/params_ref_'+c+'.pkl')
	for p in pars_select:
		table.loc[(p,'par'),c] = df.loc[p,'value']
		table.loc[(p,'se'),c] = df.loc[p,'se']


In [5]:
pars_select = ['delta_h1','delta_h2','price','tfp']
labels = ['$\\alpha_{10}$','$\\alpha_{11}$','$\\frac{p}{p_{US}}$','$\\frac{A}{A_{US}}$']
maps_labels = dict(zip(pars_select,labels))
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','se']])))

In [6]:
for c in table.columns:
	table[c] = table[c].astype('float64')
	table[c] = table[c].round(3)
	table[c] = table[c].astype('str')
	table[c] = np.where(table[c]=='nan','-',table[c])
	for s in table.index:
		if s[1]=='se':
			if table.loc[s,c]!='-':
				fmt = "({se})"
				table.loc[s,c] = fmt.format(se=table.loc[s,c])
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','']])))

In [7]:
table

us       de       dk       fr       it       nl  \
$\alpha_{10}$         -0.967   -1.296   -1.601   -1.094   -0.716   -1.265   
                     (0.161)  (0.098)   (0.01)  (0.013)  (0.032)   (0.03)   
$\alpha_{11}$          3.487    3.954    4.273    3.698    3.868    3.998   
                     (0.051)  (0.031)  (0.015)  (0.023)  (0.029)  (0.141)   
$\frac{p}{p_{US}}$       1.0    0.847    0.888    0.615    0.713    0.698   
                           -  (0.012)  (0.015)  (0.005)   (0.18)  (0.021)   
$\frac{A}{A_{US}}$       1.0    1.009    1.255    0.924    0.639     1.01   
                           -  (0.099)  (0.008)  (0.026)  (0.007)  (0.078)   

                          se       sp  
$\alpha_{10}$         -1.525   -0.006  
                     (0.011)  (0.008)  
$\alpha_{11}$          4.311    3.394  
                     (0.019)   (0.02)  
$\frac{p}{p_{US}}$     0.844     0.65  
                     (0.026)   (0.03)  
$\frac{A}{A_{US}}$     0.788    0.811  
                     (0.003)  (0.034)

In [8]:
table.to_latex('../tables/table_8_country_specific_estimates.tex')

## Average US-EU differences

This number is used in counterfactual analysis. 

In [9]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{p}{p_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.7507142857142857

In [10]:
1/prices['de':].astype('float64').mean()-1

0.33206470028544244

In [12]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{A}{A_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.9194285714285714

In [13]:
1/prices['de':].astype('float64').mean()-1

0.08763206960845249